In [14]:
import pandas as pd
import re
import requests
from tqdm import tqdm

df = pd.read_csv("./올영_세일.csv")

def extract_goods_no(url):
    m = re.search(r"goodsNo=([A-Za-z0-9]+)", url)
    return m.group(1) if m else None

df["goodsNo"] = df["url"].apply(extract_goods_no)

def get_review_data(goods_no):
    url = "https://www.oliveyoung.co.kr/store/goods/getGoodsEvalList.do"
    payload = {
        "goodsNo": goods_no,
        "pageIdx": 1,
        "pageSize": 10,
        "sort": "01",
        "filterVal": "",
    }
    try:
        r = requests.post(url, data=payload)
        j = r.json()
        return j["totalCount"], j["avgScore"]
    except:
        return None, None

review_counts = []
review_scores = []

for goods_no in tqdm(df["goodsNo"]):
    total, score = get_review_data(goods_no)
    review_counts.append(total)
    review_scores.append(score)

df["review_count"] = review_counts
df["review_score"] = review_scores

df.to_csv("올영_세일_리뷰추가.csv", index=False, encoding="utf-8-sig")

print("완료!")

  0%|          | 0/1200 [00:00<?, ?it/s]

 34%|███▍      | 406/1200 [00:19<00:38, 20.45it/s]

#### 공통: 상품 목록 준비

In [1]:
import pandas as pd
import re

# 세일 상품 데이터
df_prod = pd.read_csv("./올영_세일.csv", encoding="utf-8")

# URL에서 goodsNo 추출
def extract_goods_no(url):
    m = re.search(r"goodsNo=([A-Za-z0-9]+)", str(url))
    return m.group(1) if m else None

df_prod["goodsNo"] = df_prod["url"].apply(extract_goods_no)

# 브랜드 컬럼 (위에서 이미 썼던 방식)
df_prod["brand"] = df_prod["name"].apply(lambda x: str(x).split("\n")[0].split()[0])

df_prod.head()

,page,name,price,orig_price,url,goodsNo,brand
0,1,메디힐\n[2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 10+...,"9,900원 ~","20,000원 ~",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000223414,메디힐
1,1,일리윤\n[어워즈특가] 일리윤 세라마이드 아토 로션 600+334ML 어워즈 한정기획,"24,500원","39,500원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238713,일리윤
2,1,에스트라\n[어워즈특가] 에스트라 아토베리어365 크림 80ml 어워즈 한정기획 (...,"24,700원","33,000원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238634,에스트라
3,1,토리든\n[어워즈특가/1등 세럼] 토리든 다이브인 저분자 히알루론산 세럼 100ml...,"33,250원","50,000원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238213,토리든
4,1,메디큐브 에이지알\n[어워즈특가] 메디큐브 에이지알 부스터프로 미니플러스 짱구 에디...,"115,900원 ~","126,000원 ~",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000237901,메디큐브


#### 리뷰 텍스트 최대 1000개 수집 (API)

In [2]:
import requests
from time import sleep

def fetch_reviews_one_goods(goods_no, max_reviews=1000, page_size=50, sleep_sec=0.2):
    """
    특정 goodsNo에 대해 최대 max_reviews개까지 리뷰를 가져와서
    리스트[dict, ...] 로 반환
    """
    url = "https://www.oliveyoung.co.kr/store/goods/getGoodsEvalList.do"
    
    all_reviews = []
    page_idx = 1
    
    while len(all_reviews) < max_reviews:
        payload = {
            "goodsNo": goods_no,
            "pageIdx": page_idx,
            "pageSize": page_size,
            "sort": "01",     # 최신순/추천순 등 (실제 값은 Network 탭 보고 맞추면 됨)
            "filterVal": ""   # 필터 없을 때
        }
        
        r = requests.post(url, data=payload)
        
        # 403 / HTML 등 들어오면 여기서 멈추고 확인 필요
        if "application/json" not in r.headers.get("Content-Type", ""):
            print(f"[{goods_no}] JSON 아님, status={r.status_code}")
            break
        
        data = r.json()
        
        # ★ 여기서 실제 응답 구조 확인 후 키 이름 정확히 맞춰야 함
        # 예시로 voList / totalCount / avgScore 가 있다고 가정
        review_list = data.get("voList") or data.get("list") or []
        if not review_list:
            break  # 더 이상 리뷰 없음
        
        for rv in review_list:
            all_reviews.append({
                "goodsNo": goods_no,
                # 점수(평점) 관련 키: 실제 JSON 보고 수정
                "score": rv.get("evalScore") or rv.get("goodsEvalScore"),
                # 리뷰 내용 관련 키: 실제 JSON 보고 수정
                "content": rv.get("evalContent") or rv.get("goodsEvalDtlCn"),
                # 작성일 관련 키: 실제 JSON 보고 수정
                "reg_dt": rv.get("regDt") or rv.get("createDt")
            })
            if len(all_reviews) >= max_reviews:
                break
        
        page_idx += 1
        sleep(sleep_sec)
    
    return all_reviews

#### 전체 상품에 대해 리뷰 수집

In [3]:
import pandas as pd
import re

# 세일 상품 CSV 불러오기
df_prod = pd.read_csv("./올영_세일_리뷰추가.csv", encoding="utf-8")

# goodsNo 추출 함수
def extract_goods_no(url):
    m = re.search(r"goodsNo=([A-Za-z0-9]+)", str(url))
    return m.group(1) if m else None

# goodsNo 컬럼 생성
df_prod["goodsNo"] = df_prod["url"].apply(extract_goods_no)

df_prod.head()

,page,name,price,orig_price,url,goodsNo,review_count,review_score
0,1,메디힐\n[2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 10+...,"9,900원 ~","20,000원 ~",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000223414,NaN,NaN
1,1,일리윤\n[어워즈특가] 일리윤 세라마이드 아토 로션 600+334ML 어워즈 한정기획,"24,500원","39,500원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238713,NaN,NaN
2,1,에스트라\n[어워즈특가] 에스트라 아토베리어365 크림 80ml 어워즈 한정기획 (...,"24,700원","33,000원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238634,NaN,NaN
3,1,토리든\n[어워즈특가/1등 세럼] 토리든 다이브인 저분자 히알루론산 세럼 100ml...,"33,250원","50,000원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238213,NaN,NaN
4,1,메디큐브 에이지알\n[어워즈특가] 메디큐브 에이지알 부스터프로 미니플러스 짱구 에디...,"115,900원 ~","126,000원 ~",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000237901,NaN,NaN


#### 리뷰 키워드 분석 / 감성 분석

#### 전처리 + 토큰 만들기 (엄청 러프한 버전)

In [6]:
import pandas as pd
import re
from collections import Counter

# 1) 리뷰 CSV 읽기 (파일 이름만 네 거에 맞추면 됨)
df_review = pd.read_csv("./올영_세일_리뷰추가.csv", encoding="utf-8")

print("컬럼 목록:", list(df_review.columns))

# 2) 텍스트가 들어있을 법한 컬럼 자동 탐색
candidates = []
for col in df_review.columns:
    cl = col.lower()
    if any(k in cl for k in ["content", "review", "text", "body", "내용", "comment"]):
        candidates.append(col)

print("텍스트 후보 컬럼:", candidates)

if not candidates:
    raise ValueError("리뷰 텍스트로 쓸 만한 컬럼을 못 찾음. df_review.columns 확인 필요.")

text_col = candidates[0]   # 제일 첫 번째 후보를 텍스트로 사용
print("사용할 텍스트 컬럼:", text_col)

# 3) content 이름으로 맞춰두기
if text_col != "content":
    df_review = df_review.rename(columns={text_col: "content"})

컬럼 목록: ['page', 'name', 'price', 'orig_price', 'url', 'goodsNo', 'review_count', 'review_score']
텍스트 후보 컬럼: ['review_count', 'review_score']
사용할 텍스트 컬럼: review_count


In [4]:
import pandas as pd

df_review = pd.read_csv("./올영_세일_리뷰추가.csv", encoding="utf-8")
# 또는 df_review = pd.read_csv("./올영_세일_리뷰추가.csv", encoding="utf-8")
df_review.head()

,page,name,price,orig_price,url,goodsNo,review_count,review_score
0,1,메디힐\n[2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 10+...,"9,900원 ~","20,000원 ~",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000223414,NaN,NaN
1,1,일리윤\n[어워즈특가] 일리윤 세라마이드 아토 로션 600+334ML 어워즈 한정기획,"24,500원","39,500원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238713,NaN,NaN
2,1,에스트라\n[어워즈특가] 에스트라 아토베리어365 크림 80ml 어워즈 한정기획 (...,"24,700원","33,000원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238634,NaN,NaN
3,1,토리든\n[어워즈특가/1등 세럼] 토리든 다이브인 저분자 히알루론산 세럼 100ml...,"33,250원","50,000원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238213,NaN,NaN
4,1,메디큐브 에이지알\n[어워즈특가] 메디큐브 에이지알 부스터프로 미니플러스 짱구 에디...,"115,900원 ~","126,000원 ~",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000237901,NaN,NaN


In [7]:
import re
from collections import Counter

def clean_text(text):
    text = str(text)
    text = re.sub(r"[^가-힣A-Za-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df_review["content_clean"] = df_review["content"].apply(clean_text)

def simple_tokenize(text):
    tokens = text.split()
    tokens = [t for t in tokens if len(t) >= 2]
    return tokens

df_review["tokens"] = df_review["content_clean"].apply(simple_tokenize)

#### 최빈 키워드 TOP N

In [8]:
all_tokens = []
for toks in df_review["tokens"]:
    all_tokens.extend(toks)

cnt = Counter(all_tokens)
cnt.most_common(50)  # 상위 50개

[('nan', 1200)]

#### 아주 단순한 감성 분석

In [9]:
positive_words = ["좋", "만족", "최고", "추천", "괜찮", "촉촉", "부드럽", "짱", "재구매"]
negative_words = ["별로", "실망", "최악", "짜증", "아쉬", "불편", "건조", "따갑", "자극"]

def sentiment_score(text):
    text = str(text)
    score = 0
    for w in positive_words:
        if w in text:
            score += 1
    for w in negative_words:
        if w in text:
            score -= 1
    return score

df_review["sentiment_score"] = df_review["content_clean"].apply(sentiment_score)

def label_sentiment(s):
    if s > 0:
        return "pos"
    elif s < 0:
        return "neg"
    else:
        return "neu"

df_review["sentiment_label"] = df_review["sentiment_score"].apply(label_sentiment)

df_review[["goodsNo", "content", "sentiment_score", "sentiment_label"]].head()

,goodsNo,content,sentiment_score,sentiment_label
0,A000000223414,NaN,0,neu
1,A000000238713,NaN,0,neu
2,A000000238634,NaN,0,neu
3,A000000238213,NaN,0,neu
4,A000000237901,NaN,0,neu


In [10]:
print(df_review.columns)
print(df_review.head())
print(df_review.isna().sum())

Index(['page', 'name', 'price', 'orig_price', 'url', 'goodsNo', 'content',
       'review_score', 'content_clean', 'tokens', 'sentiment_score',
       'sentiment_label'],
      dtype='object')
   page                                               name       price  \
0     1  메디힐\n[2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 10+...    9,900원 ~   
1     1    일리윤\n[어워즈특가] 일리윤 세라마이드 아토 로션 600+334ML 어워즈 한정기획     24,500원   
2     1  에스트라\n[어워즈특가] 에스트라 아토베리어365 크림 80ml 어워즈 한정기획 (...     24,700원   
3     1  토리든\n[어워즈특가/1등 세럼] 토리든 다이브인 저분자 히알루론산 세럼 100ml...     33,250원   
4     1  메디큐브 에이지알\n[어워즈특가] 메디큐브 에이지알 부스터프로 미니플러스 짱구 에디...  115,900원 ~   

   orig_price                                                url  \
0   20,000원 ~  https://www.oliveyoung.co.kr/store/goods/getGo...   
1     39,500원  https://www.oliveyoung.co.kr/store/goods/getGo...   
2     33,000원  https://www.oliveyoung.co.kr/store/goods/getGo...   
3     50,000원  https://www.oliveyoung.co.kr/store/goods/getGo...   
4  126,000원 ~  https:/

### 리뷰가 비어있었음을 확인함. 

#### 리뷰 API 호출 함수

In [ ]:
import requests

def fetch_reviews_one_goods(goods_no, max_reviews=500):
    url = "https://www.oliveyoung.co.kr/store/goods/getGoodsReview.do"
    rows = []
    page = 1

    while len(rows) < max_reviews:
        payload = {
            "goodsNo": goods_no,
            "pageIdx": page,
            "pageSize": 10
        }
        r = requests.post(url, data=payload)

        try:
            data = r.json()
        except:
            break  # JSON decoding 실패 → 종료

        review_list = data.get("goodsReviewList", [])
        if not review_list:
            break

        for rv in review_list:
            rows.append({
                "goodsNo": goods_no,
                "content": rv.get("reviewContent", ""),
                "review_score": rv.get("reviewScore", None),
                "regDt": rv.get("regDt", "")
            })

        page += 1

    return rows

#### 전체 상품에 대해 리뷰 수집 

In [15]:
from tqdm import tqdm

review_all = []

for goods_no in tqdm(df_prod["goodsNo"].dropna().unique()):
    reviews = fetch_reviews_one_goods(goods_no, max_reviews=500)
    review_all.extend(reviews)

df_review = pd.DataFrame(review_all)
df_review.to_csv("올영_세일_리뷰텍스트.csv", index=False, encoding="utf-8-sig")

df_review.head()

100%|██████████| 1200/1200 [00:54<00:00, 22.20it/s]


""


#### 리뷰가 없음을 파악하고 크롤링을 시도함 

In [17]:
import pandas as pd
import re

df_prod = pd.read_csv("./올영_세일_리뷰추가.csv", encoding="utf-8")

# goodsNo 추출 함수
def extract_goods_no(url):
    m = re.search(r"goodsNo=([A-Za-z0-9]+)", str(url))
    return m.group(1) if m else None

df_prod["goodsNo"] = df_prod["url"].apply(extract_goods_no)
df_prod = df_prod.dropna(subset=["goodsNo"]).reset_index(drop=True)

df_prod.head()

,page,name,price,orig_price,url,goodsNo,review_count,review_score
0,1,메디힐\n[2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 10+...,"9,900원 ~","20,000원 ~",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000223414,NaN,NaN
1,1,일리윤\n[어워즈특가] 일리윤 세라마이드 아토 로션 600+334ML 어워즈 한정기획,"24,500원","39,500원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238713,NaN,NaN
2,1,에스트라\n[어워즈특가] 에스트라 아토베리어365 크림 80ml 어워즈 한정기획 (...,"24,700원","33,000원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238634,NaN,NaN
3,1,토리든\n[어워즈특가/1등 세럼] 토리든 다이브인 저분자 히알루론산 세럼 100ml...,"33,250원","50,000원",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000238213,NaN,NaN
4,1,메디큐브 에이지알\n[어워즈특가] 메디큐브 에이지알 부스터프로 미니플러스 짱구 에디...,"115,900원 ~","126,000원 ~",https://www.oliveyoung.co.kr/store/goods/getGo...,A000000237901,NaN,NaN


##### 리뷰 API 호출 함수 

In [18]:
import requests

def fetch_reviews_one_goods(goods_no, max_reviews=500):
    url = "https://www.oliveyoung.co.kr/store/goods/getGoodsReview.do"

    rows = []
    page = 1

    while len(rows) < max_reviews:
        payload = {
            "goodsNo": goods_no,
            "pageIdx": page,
            "pageSize": 10
        }

        res = requests.post(url, data=payload)
        try:
            data = res.json()
        except:
            print("JSON decode 실패:", goods_no)
            break

        review_list = data.get("goodsReviewList", [])
        if not review_list:
            break

        for r in review_list:
            rows.append({
                "goodsNo": goods_no,
                "content": r.get("reviewContent", ""),
                "review_score": r.get("reviewScore", None),
                "regDt": r.get("regDt", "")
            })

        page += 1

    return rows

##### 전체 상품 리뷰 수집

In [19]:
from tqdm import tqdm

all_reviews = []

unique_goods = df_prod["goodsNo"].unique()

for goods_no in tqdm(unique_goods):
    reviews = fetch_reviews_one_goods(goods_no, max_reviews=500)
    all_reviews.extend(reviews)
    print(goods_no, "→", len(reviews), "개 수집")

df_review = pd.DataFrame(all_reviews)
df_review.to_csv("올영_세일_리뷰텍스트.csv", index=False, encoding="utf-8-sig")
df_review.head()

  0%|          | 3/1200 [00:00<01:03, 18.87it/s]

JSON decode 실패: A000000223414
A000000223414 → 0 개 수집
JSON decode 실패: A000000238713
A000000238713 → 0 개 수집
JSON decode 실패: A000000238634
A000000238634 → 0 개 수집
JSON decode 실패: A000000238213
A000000238213 → 0 개 수집
JSON decode 실패: A000000237901
A000000237901 → 0 개 수집


  1%|          | 9/1200 [00:00<00:59, 19.92it/s]

JSON decode 실패: A000000238402
A000000238402 → 0 개 수집
JSON decode 실패: A000000239080
A000000239080 → 0 개 수집
JSON decode 실패: A000000205979
A000000205979 → 0 개 수집
JSON decode 실패: A000000238801
A000000238801 → 0 개 수집
JSON decode 실패: A000000238523
A000000238523 → 0 개 수집


  1%|▏         | 15/1200 [00:00<00:53, 22.03it/s]

...

100%|█████████▉| 1197/1200 [00:52<00:00, 23.72it/s]

JSON decode 실패: A000000184658
A000000184658 → 0 개 수집
JSON decode 실패: A000000192105
A000000192105 → 0 개 수집
JSON decode 실패: A000000139331
A000000139331 → 0 개 수집
JSON decode 실패: A000000204431
A000000204431 → 0 개 수집
JSON decode 실패: A000000209139
A000000209139 → 0 개 수집


100%|██████████| 1200/1200 [00:52<00:00, 22.64it/s]

JSON decode 실패: A000000190915
A000000190915 → 0 개 수집


""


##### 리뷰가 수집이 안 된 것을 확인하여 페이지 내부에 '리뷰'가 있는지를 확인함 

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

def debug_review_list(goods_no):
    print("=== DEBUG 리뷰 리스트 시작:", goods_no, "=== ")

    options = Options()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goods_no}"
    driver.get(url)
    time.sleep(2)

    # 리뷰 탭 클릭
    tab = driver.find_element(By.XPATH, "//*[contains(text(), '리뷰')]")
    driver.execute_script("arguments[0].click();", tab)
    time.sleep(2)

    # 스크롤 여러 번 내려주기
    for _ in range(8):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1)

    # 내가 추정한 리뷰 박스 셀렉터
    review_blocks = driver.find_elements(By.CSS_SELECTOR, "div.css-13jv0xs")
    print("리뷰 박스 개수:", len(review_blocks))

    for i, rb in enumerate(review_blocks[:5]):
        print(f"[리뷰 {i}] -------------------------")
        print(rb.text[:300])

    driver.quit()
    print("=== DEBUG 끝 ===")

# 여기까지가 함수 정의, 아래 한 줄이 실행
debug_review_list("A000000223414")

=== DEBUG 리뷰 리스트 시작: A000000223414 === 
리뷰 박스 개수: 0
=== DEBUG 끝 ===


#### 오류로 인하여 브라우저 안에서 JS로 리뷰 API 호출” → JSON 그대로 받아옴

###### 한 상품만 test

In [27]:
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def fetch_reviews_one_goods_via_js(goods_no, max_pages=5, page_size=20):
    """
    Selenium으로 상품 상세 페이지를 연 다음,
    페이지 안에서 fetch()로 getGoodsReview.do를 호출해서
    리뷰 JSON을 그대로 받아오는 함수.
    """
    options = Options()
    # options.add_argument("--headless=new")  # 필요하면 주석 해제
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # 상품 상세 페이지 열어서 Cloudflare / 세션 통과
    detail_url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goods_no}"
    driver.get(detail_url)
    time.sleep(3)

    all_rows = []

    for page in range(1, max_pages + 1):
        script = """
        var goodsNo = arguments[0];
        var pageIdx = arguments[1];
        var pageSize = arguments[2];
        var callback = arguments[3];

        fetch("https://www.oliveyoung.co.kr/store/goods/getGoodsReview.do", {
            method: "POST",
            headers: {
                "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
            },
            body: "goodsNo=" + goodsNo + "&pageIdx=" + pageIdx + "&pageSize=" + pageSize
        })
        .then(res => res.json())
        .then(data => {
            callback(JSON.stringify(data));
        })
        .catch(err => {
            callback(JSON.stringify({error: String(err)}));
        });
        """

        raw = driver.execute_async_script(script, goods_no, page, page_size)
        data = json.loads(raw)

        # 에러나면 중단
        if "error" in data:
            print("JS fetch 에러:", goods_no, data["error"])
            break

        reviews = data.get("goodsReviewList", [])
        if not reviews:
            break

        for r in reviews:
            all_rows.append({
                "goodsNo": goods_no,
                "content": r.get("reviewContent", ""),
                "score": r.get("reviewScore", None),
                "regDt": r.get("regDt", "")
            })

    driver.quit()
    return all_rows

In [28]:
rows_test = fetch_reviews_one_goods_via_js("A000000223414", max_pages=3, page_size=20)
len(rows_test), rows_test[:3]

JS fetch 에러: A000000223414 SyntaxError: Unexpected token '<', "




<!DOCTYPE "... is not valid JSON


(0, [])

#### 오류로 인하여  HTML 안에 있는 “preloaded JSON” 을 추출

##### 리뷰 데이터가 포함된 스크립트 추출 코드

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import re
import json
import time

def extract_review_json(goods_no):
    options = Options()
    options.add_argument("--headless=new")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goods_no}"
    driver.get(url)
    time.sleep(3)

    html = driver.page_source

    # 1) "__PRELOADED_STATE__" 또는 유사 키 찾기
    m = re.search(r"__PRELOADED_STATE__\s*=\s*(\{.*?\});", html, re.S)
    if not m:
        print("❌ 리뷰 JSON 없음:", goods_no)
        return None

    json_text = m.group(1)

    try:
        data = json.loads(json_text)
        return data
    except Exception as e:
        print("❌ JSON decode 실패:", e)
        return None

# 실행 테스트
data = extract_review_json("A000000238713")
data

❌ 리뷰 JSON 없음: A000000238713


In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def get_reviews(goods_no, max_scroll=50):
    url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goods_no}"
    
    options = Options()
    options.add_argument("--window-size=1280,1000")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get(url)
    time.sleep(3)

    review_list = []

    last_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(max_scroll):
        # 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)

        # 리뷰 DOM 긁기 (class 이름만 실제 페이지 보고 맞추면 됨)
        reviews = driver.find_elements(By.CSS_SELECTOR, ".review-cont")   # ← 너가 class 명 맞춰서 수정!

        for r in reviews:
            try:
                txt = r.text.strip()
                if txt and txt not in review_list:
                    review_list.append(txt)
            except:
                pass

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.quit()
    return review_list

# 테스트
goods_no = "A000000238713"
reviews = get_reviews(goods_no, max_scroll=40)

print(len(reviews))
for r in reviews[:5]:
    print("------")
    print(r)

0


#### 오류 확인 후 리뷰 API 후보 URL 자동 추출 스크립트 (Selenium + Performance Log 방식)

In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json
import time

def find_review_api(goods_no):
    options = Options()
    options.add_argument("--auto-open-devtools-for-tabs")  # 네트워크 로그 활성화
    options.set_capability("goog:loggingPrefs", {"performance": "ALL"})

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goods_no}"
    driver.get(url)
    time.sleep(5)  # 네트워크 요청 발생 대기

    logs = driver.get_log("performance")

    candidate_urls = []

    for entry in logs:
        try:
            msg = json.loads(entry["message"])["message"]
            if msg["method"] == "Network.requestWillBeSent":
                req = msg["params"]["request"]
                request_url = req["url"]

                # 리뷰 관련된 단어 포함된 URL 수집
                if any(keyword in request_url.lower() for keyword in [
                    "review", "comment", "goods", "reply", "reviewlist", "photo", "rate"
                ]):
                    candidate_urls.append(request_url)

        except:
            continue

    print("\n===== 🔍 리뷰 API 후보 URL =====")
    for url in candidate_urls:
        print(url)

    print("\n총 후보 개수:", len(candidate_urls))
    driver.quit()


# ▶ 원하는 상품번호 넣고 실행
find_review_api("A000000238713")


===== 🔍 리뷰 API 후보 URL =====
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000238713
devtools://devtools/bundled/models/live-metrics/web-vitals-injected/web-vitals-injected.generated.js
https://image.oliveyoung.co.kr/cfimages/cf-goods/uploads/images/thumbnails/10/0000/0023/A00000023871314ko.jpg?l=ko&QT=85&SF=webp&sharpen=1x0.5
https://cf-static.oliveyoung.co.kr/lavender/2025120401/_next/static/chunks/app/goods/%5BgoodsNumber%5D/page-f082364fcbdd9b04.js
https://image.oliveyoung.co.kr/cfimages/cf-goods/uploads/images/thumbnails/10/0000/0023/A00000023871312ko.jpg?l=ko&QT=85&SF=webp&sharpen=1x0.5
https://www.oliveyoung.co.kr/goods/api/v1/description?goodsNumber=A000000238713
https://www.oliveyoung.co.kr/claim-front/api/v1/goods/getGoodsQnACount?goodsNo=A000000238713
https://www.oliveyoung.co.kr/goods/api/v1/extra
https://receiver.ai.oliveyoung.co.kr/rest/logs?data=%7B%22cuid%22%3A%228b47cf9f-efd1-48e4-8f83-10ee8a07945b%22%2C%22domain%22%3A%22www.oliveyoung.co.kr%22

In [33]:
def debug_review_list(goods_no):
    options = Options()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goods_no}"
    driver.get(url)
    time.sleep(2)

    # 리뷰 탭 클릭
    tab = driver.find_element(By.XPATH, "//*[contains(text(), '리뷰')]")
    driver.execute_script("arguments[0].click();", tab)
    time.sleep(2)

    # --------------------------
    # 스크롤 내리기 (중요!)
    # --------------------------
    for _ in range(8):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1)

    # 리뷰 리스트 확인
    review_blocks = driver.find_elements(By.CSS_SELECTOR, "div.css-13jv0xs")  # ← 리뷰 본문 박스
    print("리뷰 박스 개수:", len(review_blocks))

    for i, rb in enumerate(review_blocks[:5]):
        print(f"[리뷰 {i}]")
        print(rb.text[:200])
        print("----------")

    driver.quit()

In [34]:
import requests

url = "https://m.oliveyoung.co.kr/review/api/v2/reviews"

payload = {
    "goodsNumber": "A000000238713",   # ← 여기만 바꾸면 다른 상품 리뷰 가능
    "pageNumber": 1,
    "pageSize": 20,
    "sort": "DATE_DESC",              # 최신순
    "isPhotoOnly": False,
}

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X)",
    "Content-Type": "application/json",
    "Accept": "application/json",
}

res = requests.post(url, json=payload, headers=headers)

data = res.json()
print(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
import requests

s = requests.Session()

# --- 여기에 네 쿠키 문자열 붙여넣기 ---
cookie_raw = "네가_복사한_cookie_전체"

# 세미콜론 기준으로 쪼개서 session.cookies에 삽입
for c in cookie_raw.split(";"):
    if "=" in c:
        key, value = c.strip().split("=", 1)
        s.cookies.set(key, value)

# 실제 요청
url = "https://m.oliveyoung.co.kr/review/api/v2/reviews"

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X)",
    "Content-Type": "application/json;charset=UTF-8",
    "Accept": "application/json",
}

payload = {
    "goodsNumber": "A000000238713",
    "pageNumber": 1,
    "pageSize": 20,
    "sort": "DATE_DESC",
    "isPhotoOnly": False,
}

res = s.post(url, json=payload, headers=headers)

print(res.status_code)
print(res.text[:500])

403
<!DOCTYPE html>
<html lang="ko">
  <head>
    <meta charset="UTF-8" />
    <meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
    <meta http-equiv="Pragma" content="no-cache" />
    <meta http-equiv="Cache-Control" content="no-cache" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <meta name="format-detection" content="telephone=no" />
    <meta
      name="viewport"
      content="width=device-width, user-scalable=no, initial-scale=1.0, maximum-scale=1.0, mi


#### 리뷰 API 요청의 Request Headers 전체

In [41]:
import requests
import json

url = "https://m.oliveyoung.co.kr/review/api/v2/reviews"

headers = {
    "User-Agent": "여기에 복사한 UA",
    "Accept": "여기에 Accept",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://m.oliveyoung.co.kr",
    "Referer": "https://m.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000238713",
    "Cookie": "네가 DevTools에서 복사한 Cookie",
}

payload = {
    "goodsNumber": "A000000238713",
    "pageNumber": 1,
    "pageSize": 10,
    "sort": "MOST_HELPFUL",
    "isPhotoOnly": False
}

res = requests.post(url, headers=headers, json=payload)

print(res.status_code)
print(res.text)

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-2: ordinal not in range(256)

In [44]:
import requests

url = "https://m.oliveyoung.co.kr/review/api/v2/reviews"

headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 10; SM-G981N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36",
    "Origin": "https://m.oliveyoung.co.kr",
    "Referer": "https://m.oliveyoung.co.kr",
    "Content-Type": "application/x-www-form-urlencoded",
}

payload = {
    "goodsNumber": "A000000238713",
    "page": 0,
    "size": 10,
    "sortType": "USEFUL_SCORE_DESC",
    "reviewType": "ALL"
}

res = requests.post(url, headers=headers, data=payload)

print("STATUS:", res.status_code)
print(res.text[:2000])

STATUS: 403
<!DOCTYPE html>
<html lang="ko">
  <head>
    <meta charset="UTF-8" />
    <meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
    <meta http-equiv="Pragma" content="no-cache" />
    <meta http-equiv="Cache-Control" content="no-cache" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <meta name="format-detection" content="telephone=no" />
    <meta
      name="viewport"
      content="width=device-width, user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0"
    />
    <title>잠시만 기다려 주세요 - 올리브영</title>
    <style>
      * {
        margin: 0;
        padding: 0;
        box-sizing: border-box;
      }

      html,
      body {
        height: 100%;
        overflow-x: hidden;
      }

      body {
        font-family: "Apple SD Gothic Neo", -apple-system, BlinkMacSystemFont,
          "Segoe UI", sans-serif;
        background: #ffffff;
        width: 100%;
        margin: 0 auto;
        position: relative;
        disp

In [45]:
import requests
import json

url = "https://m.oliveyoung.co.kr/review/api/v2/reviews"

headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 10; SM-G973N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.230 Mobile Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://m.oliveyoung.co.kr",
    "Referer": "https://m.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000238713",
    "Sec-Fetch-Site": "same-site",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Dest": "empty",
}

payload = {
    "goodsNumber": "A000000238713",
    "page": 0,
    "size": 10,
    "sortType": "USEFUL_SCORE_DESC",
    "reviewType": "ALL",
}

res = requests.post(url, headers=headers, data=json.dumps(payload))

print("STATUS:", res.status_code)
print(res.text[:1000])

STATUS: 403
<!DOCTYPE html>
<html lang="ko">
  <head>
    <meta charset="UTF-8" />
    <meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
    <meta http-equiv="Pragma" content="no-cache" />
    <meta http-equiv="Cache-Control" content="no-cache" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <meta name="format-detection" content="telephone=no" />
    <meta
      name="viewport"
      content="width=device-width, user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0"
    />
    <title>잠시만 기다려 주세요 - 올리브영</title>
    <style>
      * {
        margin: 0;
        padding: 0;
        box-sizing: border-box;
      }

      html,
      body {
        height: 100%;
        overflow-x: hidden;
      }

      body {
        font-family: "Apple SD Gothic Neo", -apple-system, BlinkMacSystemFont,
          "Segoe UI", sans-serif;
        background: #ffffff;
        width: 100%;
        margin: 0 auto;
        position: relative;
        disp

#### 브랜드별 리뷰량·평점 지표 생성

In [16]:
# 리뷰 DF + 상품 DF를 goodsNo 기준으로 병합
df_rev_merged = df_review.merge(
    df_prod[["goodsNo", "brand", "name", "price_num", "orig_price_num", "discount_rate"]],
    on="goodsNo",
    how="left"
)

KeyError: "['brand', 'price_num', 'orig_price_num', 'discount_rate'] not in index"

#### 브랜드별 집계

In [ ]:
brand_review_stats = df_rev_merged.groupby("brand").agg(
    n_reviews = ("goodsNo", "count"),
    avg_score = ("score", "mean"),
    pos_ratio = ("sentiment_label", lambda x: (x == "pos").mean()),
    neg_ratio = ("sentiment_label", lambda x: (x == "neg").mean())
).sort_values("n_reviews", ascending=False)

brand_review_stats.head(30)

#### 리뷰 많은 상품만 필터링해서 모델 강화

In [ ]:
# 상품별 리뷰 집계
prod_review_stats = df_rev_merged.groupby("goodsNo").agg(
    n_reviews = ("goodsNo", "count"),
    avg_review_score = ("score", "mean"),
    avg_sentiment = ("sentiment_score", "mean")
).reset_index()

df_prod2 = df_prod.merge(prod_review_stats, on="goodsNo", how="left")

# 결측치 0/중립 처리
df_prod2["n_reviews"] = df_prod2["n_reviews"].fillna(0)
df_prod2["avg_review_score"] = df_prod2["avg_review_score"].fillna(df_prod2["avg_review_score"].mean())
df_prod2["avg_sentiment"] = df_prod2["avg_sentiment"].fillna(0)

In [47]:
from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    browser = p.webkit.launch(headless=False)
    context = browser.new_context(
        user_agent="Mozilla/5.0 (Linux; Android 10; SM-G973N) AppleWebKit/537.36 Chrome/120 Mobile Safari/537.36"
    )
    page = context.new_page()

    # 상품 상세 페이지 먼저 접속 → Cloudflare 통과
    page.goto("https://m.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000238713")

    # 리뷰 API 직접 실행
    api_res = page.request.post(
        "https://m.oliveyoung.co.kr/review/api/v2/reviews",
        data={
            "goodsNumber": "A000000238713",
            "page": 0,
            "size": 10,
            "sortType": "USEFUL_SCORE_DESC",
            "reviewType": "ALL",
        }
    )

    print(api_res.json())

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [54]:
import asyncio
from playwright.async_api import async_playwright

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # ★ chromium으로 변경 ★

        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Linux; Android 10; SM-G973N) AppleWebKit/537.36 Chrome/120 Mobile Safari/537.36"
        )
        page = await context.new_page()

        # Cloudflare 우회 겸 렌더링 기다림
        await page.goto("https://m.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000238713")
        await page.wait_for_timeout(3000)

        api_res = await page.request.post(
            "https://m.oliveyoung.co.kr/review/api/v2/reviews",
            data={
                "goodsNumber": "A000000238713",
                "page": 0,
                "size": 10,
                "sortType": "USEFUL_SCORE_DESC",
                "reviewType": "ALL",
            }
        )

        print("STATUS:", api_res.status)
        print(await api_res.json())

        await browser.close()

In [55]:
await run()

STATUS: 200
{'status': 'SUCCESS', 'code': 200, 'message': '요청에 성공하였습니다.', 'pagination': None, 'data': [{'reviewId': 51385328, 'content': '향은 무향에 가깝고 발림성도 좋고 끈적임도 없어서 좋은데 보습력은 다소 아쉽네요ㅠㅠ', 'goodsDto': {'goodsNumber': 'A000000157820', 'itemNumber': '001', 'legacyGoodsNumber': '8809685742531', 'goodsName': '[4년연속1위/100시간보습]\xa0일리윤 세라마이드 아토 로션 334ML+100ML(기획/단품)', 'optionName': '334ML+100ML'}, 'reviewScore': 4, 'hasPhoto': True, 'isRepurchase': False, 'isMonthUseReview': False, 'isMonthOverReview': True, 'reviewType': 'NORMAL', 'usefulPoint': 5436.0, 'photoReviewList': [{'imageSequence': 1, 'imagePath': '2025/11/14/6f7e9e0ac470467d9ec9107b7d86f86a1763089156522.png'}, {'imageSequence': 2, 'imagePath': '2025/11/14/d022afe56a274e91bc04c251217d01261763089163881.png'}, {'imageSequence': 3, 'imagePath': '2025/11/14/7d8a2f7346724f5db8af96cce3a3dbb81763089170388.png'}], 'profileDto': {'memberNickname': 'sol솔', 'profileImageUrl': '2023/01/06/1672991954674.png', 'isShutterbrity': False, 'isTopReviewe

In [56]:
import asyncio
from playwright.async_api import async_playwright
import re

# ▼ (1) 상품번호 자동 추출 함수
def extract_goods_no(url):
    match = re.search(r"goodsNo=([A-Z0-9]+)", url)
    return match.group(1) if match else None

# ▼ (2) 메인 실행 함수
async def run():
    PRODUCT_URL = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000238713&dispCatNo=90000010009"
    GOODS_NO = extract_goods_no(PRODUCT_URL)

    print("상품번호:", GOODS_NO)

    async with async_playwright() as p:
        # 안드로이드 UA → 올리브영이 신뢰함
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Linux; Android 10; SM-G973N) AppleWebKit/537.36 Chrome/120 Mobile Safari/537.36"
        )
        page = await context.new_page()

        # 리뷰 API 엔드포인트
        api_url = "https://m.oliveyoung.co.kr/review/api/v2/reviews"

        # payload = form-data 방식 (중요!)
        payload = {
            "goodsNumber": GOODS_NO,
            "page": 0,
            "size": 10,
            "sortType": "USEFUL_SCORE_DESC",
            "reviewType": "ALL"
        }

        # 필수 헤더 3개만 넣기
        headers = {
            "User-Agent": "Mozilla/5.0 (Linux; Android 10; SM-G973N) AppleWebKit/537.36 Chrome/120 Mobile Safari/537.36",
            "Origin": "https://m.oliveyoung.co.kr",
            "Referer": "https://m.oliveyoung.co.kr/",
        }

        # API 요청 보내기
        response = await page.request.post(api_url, headers=headers, data=payload)

        print("STATUS:", response.status)
        data = await response.json()
        print(data)

        await browser.close()

# ▼ (3) Jupyter에서 실행
await run()

상품번호: A000000238713
STATUS: 200
{'status': 'SUCCESS', 'code': 200, 'message': '요청에 성공하였습니다.', 'pagination': None, 'data': [{'reviewId': 51385328, 'content': '향은 무향에 가깝고 발림성도 좋고 끈적임도 없어서 좋은데 보습력은 다소 아쉽네요ㅠㅠ', 'goodsDto': {'goodsNumber': 'A000000157820', 'itemNumber': '001', 'legacyGoodsNumber': '8809685742531', 'goodsName': '[4년연속1위/100시간보습]\xa0일리윤 세라마이드 아토 로션 334ML+100ML(기획/단품)', 'optionName': '334ML+100ML'}, 'reviewScore': 4, 'hasPhoto': True, 'isRepurchase': False, 'isMonthUseReview': False, 'isMonthOverReview': True, 'reviewType': 'NORMAL', 'usefulPoint': 5436.0, 'photoReviewList': [{'imageSequence': 1, 'imagePath': '2025/11/14/6f7e9e0ac470467d9ec9107b7d86f86a1763089156522.png'}, {'imageSequence': 2, 'imagePath': '2025/11/14/d022afe56a274e91bc04c251217d01261763089163881.png'}, {'imageSequence': 3, 'imagePath': '2025/11/14/7d8a2f7346724f5db8af96cce3a3dbb81763089170388.png'}], 'profileDto': {'memberNickname': 'sol솔', 'profileImageUrl': '2023/01/06/1672991954674.png', 'isShutterbrity': 

In [57]:
import pandas as pd

df = pd.DataFrame(data["data"])
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

df[["score", "content", "createDt"]]

TypeError: 'NoneType' object is not subscriptable

In [53]:
!playwright install

129.7 MiB [                    ] 0% 0.0s129.7 MiB [                    ] 0% 63.7s129.7 MiB [                    ] 0% 172.2s129.7 MiB [                    ] 0% 199.0s129.7 MiB [                    ] 0% 125.8s129.7 MiB [                    ] 0% 99.8s129.7 MiB [                    ] 0% 86.0s129.7 MiB [                    ] 0% 78.8s129.7 MiB [                    ] 0% 79.2s129.7 MiB [                    ] 0% 72.8s129.7 MiB [                    ] 0% 78.4s129.7 MiB [                    ] 0% 71.5s129.7 MiB [                    ] 0% 69.8s129.7 MiB [                    ] 1% 51.8s129.7 MiB [                    ] 1% 48.9s129.7 MiB [                    ] 1% 48.5s129.7 MiB [                    ] 1% 45.4s129.7 MiB [                    ] 1% 45.1s129.7 MiB [                    ] 1% 43.5s129.7 MiB [                    ] 1% 42.5s129.7 MiB [                    ] 1% 41.4s129.7 MiB [                    ] 1% 40.6s129.7 MiB [                    ] 1% 39.7s129.7 MiB [                    ] 2% 38.1s129.7 MiB [   

#### 리뷰가 어느 정도 이상 있는 상품만 사용 (예: 50개 이상)

In [ ]:
df_model = df_prod2.copy()
# 랭크 다시 정의 (혹은 기존 rank 그대로 써도 됨)
df_model["rank"] = df_model.index + 1

df_model_hr = df_model[df_model["n_reviews"] >= 50].copy()
len(df_model_hr)

#### 피처에 리뷰 관련 변수 추가하고 예측 모델 강화

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# 피처: 가격 + 할인율 + 리뷰특성
feature_cols = [
    "price_num", "orig_price_num", "discount_rate",
    "n_reviews", "avg_review_score", "avg_sentiment"
]

df_model_hr = df_model_hr.dropna(subset=feature_cols + ["rank"])
X = df_model_hr[feature_cols]
y = df_model_hr["rank"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

xgb = XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="reg:squarederror"
)

xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)

print("MAE:", mean_absolute_error(y_test, pred))
print("R2 :", r2_score(y_test, pred))

#### 어떤 변수가 제일 중요한지

In [ ]:
importances = pd.Series(xgb.feature_importances_, index=feature_cols)
importances.sort_values(ascending=False)